In [2]:
import mesh2sdf
import trimesh

filename = '../model/dragon.obj'
mesh = trimesh.load(filename, force='mesh')

In [3]:
mesh.is_watertight

False

In [4]:
mesh_scale = 0.8
size = 128#sdf's shape is (size,size,size)
level = 2 / size

In [5]:
vertices=mesh.vertices
bbmax=vertices.max(0)
bbmin=vertices.min(0)
center=(bbmin+bbmax)*0.5
print((bbmax - bbmin).max())
scale = 2.0 * mesh_scale / (bbmax - bbmin).max()
vertices = (vertices - center) * scale

1.2517269999999998


In [6]:
# fix mesh
import time
t0 = time.time()
sdf, mesh = mesh2sdf.compute(vertices, mesh.faces, size, fix=True, level=level, return_mesh=True)
t1 = time.time()

In [7]:
# output
import numpy as np
# mesh.vertices = mesh.vertices / scale + center
# mesh.export(filename[:-4] + '.fixed.obj')
# np.save(filename[:-4] + '.npy', sdf)
print('It takes %.4f seconds to process %s' % (t1-t0, filename))

It takes 16.1011 seconds to process ../model/dragon.obj


### Visualize SDF

In [8]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import skimage.measure

In [9]:
# sdf = np.load('../model/bunny.npy')
print(sdf.shape)
size = sdf.shape[0]
print(sdf.max(), sdf.min())

(128, 128, 128)
1.270579 -0.18576771


In [ ]:
mesh_scale = 0.8
levels = [-0.02, 0.0, 0.02]
# extract level sets
for i, level in enumerate(levels):
  vtx, faces, _, _ = skimage.measure.marching_cubes(sdf, level)

  vtx = vtx * (mesh_scale * 2.0 / size) - 1.0
  mesh = trimesh.Trimesh(vtx, faces)
  mesh.export(os.path.join('./', 'l%.2f.obj' % level))

In [ ]:
# draw image
for i in range(size):
  array_2d = sdf[:, :, i]

  num_levels = 6
  fig, ax = plt.subplots(figsize=(2.75, 2.75), dpi=300)
  levels_pos = np.logspace(-2, 0, num=num_levels)  # logspace
  levels_neg = -1. * levels_pos[::-1]
  levels = np.concatenate((levels_neg, np.zeros((0)), levels_pos), axis=0)
  colors = plt.get_cmap("Spectral")(np.linspace(0., 1., num=num_levels*2+1))

  sample = array_2d
  # sample = np.flipud(array_2d)
  CS = ax.contourf(sample, levels=levels, colors=colors)

  ax.contour(sample, levels=levels, colors='k', linewidths=0.1)
  ax.contour(sample, levels=[0], colors='k', linewidths=0.3)
  ax.axis('off')

  plt.savefig(os.path.join('./frames', '%03d.png' % i))